In [ ]:
!pip install newspaper3k transformers gradio --quiet 

## Load libraries

In [2]:
from newspaper import Article
from newspaper import Config
import nltk
nltk.download('punkt')

from transformers import pipeline
import gradio as gr
from gradio.mix import Parallel, Series

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = USER_AGENT
config.request_timeout = 10

url = 'https://www.technologyreview.com/2021/07/09/1028140/ai-voice-actors-sound-human/'
article = Article(url, config=config)

## Download the article

In [4]:
article.download() 

In [ ]:
article.html

## Parse information from article

In [6]:
article.parse() 

authors = ", ".join(author for author in article.authors)
title = article.title
date = article.publish_date
text = article.text
image = article.top_image
videos = article.movies
url = article.url

In [7]:
print("Information about the article")
print("=" * 30)
print(f"Title: {title}")
print(f"Author(s): {authors}")
print(f"Publish date: {date}")
print(f"Image: {image}")
print(f"Videos: {videos}")
print(f"Article link: {url}")
print(f"Content: {text[:100] + '...'}")

Information about the article
Title: AI voice actors sound more human than ever—and they’re ready to hire
Author(s): Karen Hao
Publish date: 2021-07-09 00:00:00
Image: https://wp.technologyreview.com/wp-content/uploads/2021/07/AIAudioActor-2.jpg?resize=1200,600
Videos: []
Article link: https://www.technologyreview.com/2021/07/09/1028140/ai-voice-actors-sound-human/
Content: The company blog post drips with the enthusiasm of a ’90s US infomercial. WellSaid Labs describes wh...


## NLP from article

In [8]:
article.nlp()

In [56]:
keywords = article.keywords
keywords.sort()
print(keywords)

['actors', 'ai', 'audio', 'certainly', 'clients', 'companies', 'different', 'everand', 'hire', 'human', 'ready', 'sound', 'theyre', 'voice', 'voices']


In [57]:
keywords = "\n".join(keyw for keyw in keywords)

In [58]:
print(f"Article Keywords: \n{keywords}")

Article Keywords: 
actors
ai
audio
certainly
clients
companies
different
everand
hire
human
ready
sound
theyre
voice
voices


### Newspaper library summary

In [59]:
print(f"Summary: \n{article.summary}")

Summary: 
Unlike a recording of a human voice actor, synthetic voices can also update their script in real time, opening up new opportunities to personalize advertising.
Human voice actors, in particular, have been left to wonder what this means for their livelihoods.
Part of what makes a human voice so human is its inconsistency, expressiveness, and ability to deliver the same lines in completely different styles, depending on the context.
Capturing these nuances involves finding the right voice actors to supply the appropriate training data and fine-tune the deep-learning models.
Many of its clients use the synthesized voices only in pre-production and switch to real voice actors for the final production.


In [ ]:
text

## summarize with Hugging Face and Gradio

In [ ]:
io1 = gr.Interface.load('huggingface/sshleifer/distilbart-cnn-12-6')
io2 = gr.Interface.load("huggingface/facebook/bart-large-cnn")
io3 = gr.Interface.load("huggingface/google/pegasus-xsum")  
io4 = gr.Interface.load("huggingface/sshleifer/distilbart-cnn-6-6")                   

iface = Parallel(io1, io2, io3, io4,
                 theme='huggingface', 
                 inputs = gr.inputs.Textbox(lines = 10, label="Text"))

iface.launch()

In [10]:
def extract_article_text(url):
  USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'
  config = Config()
  config.browser_user_agent = USER_AGENT
  config.request_timeout = 10

  article = Article(url, config=config)
  article.download()
  article.parse()
  text = article.text
  return text

In [ ]:
extractor = gr.Interface(extract_article_text, 'text', 'text')
summarizer = gr.Interface.load("huggingface/facebook/bart-large-cnn")

sample_url = [['https://www.technologyreview.com/2021/07/22/1029973/deepmind-alphafold-protein-folding-biology-disease-drugs-proteome/'],
              ['https://www.technologyreview.com/2021/07/21/1029860/disability-rights-employment-discrimination-ai-hiring/'],
              ['https://www.technologyreview.com/2021/07/09/1028140/ai-voice-actors-sound-human/']]

desc =  '''
        Let Hugging Face models summarize articles for you. 
        Note: Shorter articles generate faster summaries.
        This summarizer uses bart-large-cnn model by Facebook
        '''

iface = Series(extractor, summarizer, 
  inputs = gr.inputs.Textbox(
      lines = 2,
      label = 'URL'
  ),
  outputs = 'text',
  title = 'News Summarizer',
  theme = 'huggingface',
  description = desc,
  examples=sample_url)

iface.launch()